# valid set 생성

### 전체 documents에서 랜덤으로 용도에 맞는 아이템 나누기

In [33]:
import random

def split_jsonl_file(input_file, output_files, percentages):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    random.shuffle(lines)

    total_lines = len(lines)
    split_indices = [int(total_lines * p) for p in percentages]

    # 각각의 파일에 저장
    start_idx = 0
    for idx, output_file in zip(split_indices, output_files):
        with open(output_file, 'w', encoding='utf-8') as f_out:
            f_out.writelines(lines[start_idx:start_idx + idx])
            start_idx += idx

# 사용 예시
input_file = './data/documents.jsonl'  # 원본 JSONL 파일 경로
output_files = ['./data/documents_25.jsonl', './data/documents_70.jsonl', './data/documents_5.jsonl']  # 출력 파일 경로
percentages = [0.25, 0.7, 0.05]  # 비율

split_jsonl_file(input_file, output_files, percentages)

### 각 용도에 맞는 질문 생성

In [ ]:
import transformers
import torch

model_id = "rtzr/ko-gemma-2-9b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

In [35]:
def request_prompt(message, max_new_tokens=100, temperature=None):
    with torch.no_grad():  # cuda 메모리 확보하기 위해 
        try:
            prompt = pipeline.tokenizer.apply_chat_template(
                message, 
                tokenize=False, 
                add_generation_prompt=True
            )

            terminators = [
                pipeline.tokenizer.eos_token_id,
                pipeline.tokenizer.convert_tokens_to_ids("<end_of_turn>")
            ]

            do_sample = True if temperature else False
            
            outputs = pipeline(
                prompt,
                max_new_tokens=max_new_tokens,
                eos_token_id=terminators,
                do_sample=do_sample,
                temperature=temperature,
            )
            return outputs[0]["generated_text"][len(prompt):]
        except Exception as e:
            print(e)
            return ""

In [ ]:
persona_normal_single_turn = """당신은 과학 기술에 특화된 상식 RAG 시스템입니다.
참조 문서를 보고, Validation Set으로 사용할 user 대화를 1개 생성해주세요.
**출력 예시**: 다음의 JSON 포맷으로 작성해주세요.
   - {"msg": [{"role": "user", "content": "솔레노이드가 뭐야?"}]}
   - {"msg": [{"role": "user", "content": "다양한 책을 catalog화 하는 코드에서 class 정의 방법 알려줘."}]}
   - {"msg": [{"role": "user", "content": "여행을 통해 얻을 수 있는 좋은 점들이 뭐가 있어?"}]}
**주의사항**:
   - 응답은 반드시 예시처럼 **JSON Body 만** 출력하세요.
"""
print(persona_normal_single_turn)

In [ ]:
persona_normal_triple_turn = """당신은 과학 기술에 특화된 상식 RAG 시스템입니다.
참조 문서를 보고, Validation Set으로 사용할 user-assistant-user 3turn 대화를 1개 생성해주세요.
**출력 예시**: 다음의 JSON 포맷으로 작성해주세요.
   - {"msg": [{"role": "user", "content": "Python 공부중이야."}, {"role": "assistant", "content": "네 python은 배워 두시면 유용합니다."}, {"role": "user", "content": "list의 값중 가장 작은 값을 알려주는 함수가 뭐야?"}]}
   - {"msg": [{"role": "user", "content": "동물들이 종종 집단으로 이주하는 경우가 발생하더라구?"}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "이렇게 집단으로 이주하게 되는 계기는 어떤 것들이 있어?"}]}
   - {"msg": [{"role": "user", "content": "은하들이 점점 멀어지고 있다던데 맞아?"}, {"role": "assistant", "content": "네 맞아요."}, {"role": "user", "content": "어떤 물리적인 현상을 보고 그걸 알게 되었어?"}]}
**주의사항**:
   - 응답은 반드시 예시처럼 **JSON Body 만** 출력하세요.
"""
print(persona_normal_triple_turn)

In [ ]:
persona_synonyms_single_turn = """당신은 과학 기술에 특화된 상식 RAG 시스템입니다.
참조 문서를 보고, Validation Set으로 사용할 user 대화를 1개 생성해주세요.
생성 대화의 핵심 단어는 **비슷한 단어**로 바꿔야 합니다.
**단어 변경 예시**:
1.
-before: 시냅스 간극에서 효소가 변성되면 아세틸콜린의 생명주기는 어떻게 변화할까요?
-after: 신경 접합부에서 효소가 변성되면 아세틸콜린의 생명주기는 어떻게 변화할까요?
2.
-before: beaver 개체수 감소가 생태계에 미치는 영향은 무엇일까요?
-after: 비버 개체수 감소가 생태계에 미치는 영향은 무엇일까요?
3.
-before: 인터넷 프록시가 하는 일을 간단하게 설명해주세요.
-after: 인터넷 Proxy가 하는 일을 간단하게 설명해주세요.
4.
-before: 고정행동양식이란 무엇이며, 예시를 들어 설명해주세요.
-after: 고정행동패턴이란 무엇이며, 예시를 들어 설명해주세요.
5.
-before: 래브라도 리트리버의 유전형이 Bbee라면 털 색깔은 어떻게 될까요?
-after: 랩래드도 리트리버의 유전형이 Bbee라면 털 색깔은 어떻게 될까요?
**출력 예시**: 다음의 JSON 포맷으로 작성해주세요.
   - {"msg": [{"role": "user", "content": "바닷물 말고 담수가 가장 많이 있는 곳이 어디야?"}]}
**주의사항**:
   - 응답은 반드시 예시처럼 **JSON Body 만** 출력하세요.
"""
print(persona_synonyms_single_turn)

In [ ]:
persona_smalltalk_single_turn = """당신은 잡담 챗봇입니다.
Validation Set으로 사용할 user 1turn 대화를 1개 생성해주세요. 요청받은 주제로 가벼운 대화를 생성합니다. 
**출력 예시**: 다음의 JSON 포맷으로 작성해주세요.
   - {"msg": [{"role": "user", "content": "니가 모르는 건 뭐야?"}]}
   - {"msg": [{"role": "user", "content": "이제 자야겠어. 또 보자."}]}
   - {"msg": [{"role": "user", "content": "날씨가 너무 더운데 무서운 얘기 좀 해줘."}]}
**주의사항**:
   - 응답은 반드시 예시처럼 **JSON Body 만** 출력하세요.
"""
print(persona_smalltalk_single_turn)

In [40]:
import json
import re

def extract_fields(response):
    # 불필요한 부분 제거
    clean_response = re.sub(r'```json|```', '', response).strip()
    
    # 마지막 쉼표 제거
    clean_response = re.sub(r',\s*}$', '}', clean_response)

    # 추가적인 공백 제거
    clean_response = re.sub(r'\s+', ' ', clean_response)

    try:
        # JSON 파싱 시도
        result = json.loads(clean_response)
        result = result["msg"]
        return result
    except json.JSONDecodeError as e:
        # 파싱 실패 시 에러 위치 출력 및 처리
        print(f"JSON 파싱 실패({e.pos}): {e},")
        raise e
    except Exception as e:
        print(f"No Json Format: {e}")
        raise e

In [41]:
import json
import gc

def generate_data(input_filename, output_filename, persona, temperature):
    print(f">> {input_filename}")

    with open(input_filename) as f, open(output_filename, "w") as of:
        idx = 0

        for line in f:
            # print(f'[{idx}]')
            # if idx > 5:
            #   break
            
            j = json.loads(line)
            document = j["content"]
            # print(document)
            prompt = [{"role": "system", "content": persona}] + [{"role": "user", "content": document}]
            # print(f'prompt : {prompt}')
            response = request_prompt(prompt, 500, temperature)
            # print(f'plain : {response}')

            try:
                msg = extract_fields(response)
                output = {
                    "valid_id": idx,
                    "docid": j["docid"],
                    "msg": msg,
                }
                print(f'{json.dumps(output, ensure_ascii=False)}')

                # 수정된 JSON을 출력 파일에 저장
                of.write(f'{json.dumps(output, ensure_ascii=False)}\n')

            except Exception as e:
                id = j["docid"]
                print(f"++ Error! {id} : {e}")
                print(f'plain response : {response}')                

            idx += 1

            del response
            gc.collect()
            torch.cuda.empty_cache()

In [78]:
# import json
# import gc

# def generate_smalltalk_data(output_filename, item_size, persona, subjects, temperature):
#     print(f">> generate_smalltalk_data")

#     with open(output_filename, "w") as of:
#         for i in range(item_size):
#             subject = random.choice(subjects)
#             print(f'subject : {subject}')
#             prompt = [{"role": "system", "content": persona}] + [{"role": "user", "content": f"**주제**: {subject}"}]
#             # print(f'prompt : {prompt}')
#             response = request_prompt(prompt, 200, temperature)
#             print(f'plain : {response}')

#             try:
#                 msg = extract_fields(response)
#                 output = {
#                     "valid_id": i,
#                     "docid": "",
#                     "msg": msg,
#                 }
#                 print(f'{json.dumps(output, ensure_ascii=False)}')

#                 # 수정된 JSON을 출력 파일에 저장
#                 of.write(f'{json.dumps(output, ensure_ascii=False)}\n')

#             except Exception as e:
#                 print(f"++ Error! {e}")
#                 print(f'plain response : {response}')                

#             del response
#             gc.collect()
#             torch.cuda.empty_cache()

In [80]:
import json
import gc

def generate_smalltalk_data(output_filename, item_size, persona, subjects, temperature):
    print(f">> generate_smalltalk_data")
    
    generated_data = []
    content_set = set()  # 중복을 확인할 content set

    # 데이터를 먼저 메모리 상에 생성
    for i in range(item_size):
        subject = random.choice(subjects)
        print(f'subject : {subject}')
        prompt = [{"role": "system", "content": persona}] + [{"role": "user", "content": f"**주제**: {subject}"}]
        response = request_prompt(prompt, 200, temperature)
        print(f'plain : {response}')

        try:
            msg = extract_fields(response)
            content = msg[0].get('content', '')  # msg에서 content를 추출

            if content in content_set:
                print(f'Duplicate content found, skipping: {content}')
                continue  # 중복된 content가 있으면 해당 항목은 건너뜀

            content_set.add(content)  # 새로운 content는 set에 추가
            generated_data.append({
                "valid_id": i,  # 일단 초기 valid_id로 추가
                "docid": "",
                "msg": msg,
            })

        except Exception as e:
            print(f"++ Error! {e}")
            print(f'plain response : {response}')                

        del response
        gc.collect()
        torch.cuda.empty_cache()

    # 중복을 제거한 후 valid_id를 재정의하면서 파일로 출력
    with open(output_filename, "w", encoding="utf-8") as of:
        for new_id, item in enumerate(generated_data):
            item['valid_id'] = new_id  # valid_id를 새로 정의
            of.write(f'{json.dumps(item, ensure_ascii=False)}\n')
    
    print(f"Data generation complete. Total unique items: {len(generated_data)}")

In [ ]:
from datetime import datetime
import time
from zoneinfo import ZoneInfo

current_time = datetime.fromtimestamp(time.time(), tz=ZoneInfo("Asia/Seoul")).strftime("%m%d-%H%M")

sigle_turn_synonyms_valid = f"./data/valid_1turn_synonyms_{current_time}.jsonl"
sigle_turn_normal_valid = f"./data/valid_1turn_{current_time}.jsonl"
triple_turn_normal_valid = f"./data/valid_3turn_{current_time}.jsonl"
single_turn_samlltalk_valid = f"./data/valid_1turn_smalltalk_{current_time}.jsonl"

sigle_turn_synonyms_documents = "./data/documents_70_sample.jsonl"
sigle_turn_normal_documents = "./data/documents_25_sample.jsonl"
triple_turn_normal_documents = "./data/documents_5_sample.jsonl"

# generate_data(sigle_turn_synonyms_documents, sigle_turn_synonyms_valid, persona_synonyms_single_turn, 0.7)
# generate_data(sigle_turn_normal_documents, sigle_turn_normal_valid, persona_normal_single_turn, None)
# generate_data(triple_turn_normal_documents, triple_turn_normal_valid, persona_normal_triple_turn, None)

subjects = ["당신에 대한 질문", "상대방에 대한 질문", "대화 종료", "일상 대화", "감정 표현"]
generate_smalltalk_data(single_turn_samlltalk_valid, 150, persona_smalltalk_single_turn, subjects, 0.5)

# sigle_turn_synonyms_documents = "./data/documents_70.jsonl"
# sigle_turn_normal_documents = "./data/documents_25.jsonl"
# triple_turn_documents = "./data/documents_5.jsonl"
# generate_data(sigle_turn_synonyms_documents, sigle_turn_synonyms, persona_synonyms_single_turn, 0.7)
# generate_data(sigle_turn_normal_documents, sigle_turn_normal, persona_normal_single_turn, None)
# generate_data(triple_turn_normal_documents, triple_turn_normal, persona_normal_triple_turn, None)

### valid.jsonl 생성

##### test set(가답안) 비율 확인

In [ ]:
import json
from collections import defaultdict, Counter

documents_file = './data/documents.jsonl'
eval_file = './data/eval.jsonl'
gt_items_file = './voting/1019-2137_voted_items(9023-8962-8689).csv'

# 1. Load the documents from each JSONL file
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

documents = load_jsonl(documents_file)
eval_data = load_jsonl(eval_file)
gt_items = load_jsonl(gt_items_file)

# Convert documents and eval_data to dict for faster lookup
documents_dict = {doc['docid']: doc for doc in documents}
eval_dict = {e['eval_id']: e for e in eval_data}

# 1. Find matching topk first docid with documents.jsonl docid and merge fields
merged_items = []
for item in gt_items:
    eval_id = item['eval_id']
    if item['topk']:  # topk might be empty
        first_docid = item['topk'][0]
        if first_docid in documents_dict:
            merged_item = {**documents_dict[first_docid], 'eval_id': eval_id}
            merged_items.append(merged_item)

# 2. Merge with eval_data based on eval_id
final_items = []
for item in merged_items:
    eval_id = item['eval_id']
    if eval_id in eval_dict:
        final_item = {**item, 'msg': eval_dict[eval_id].get('msg', [])}  # merge msg if exists
        final_items.append(final_item)

# 3. Remove 'topk' field
for item in final_items:
    if 'topk' in item:
        del item['topk']

# 4. Group by 'src' and calculate the ratio of items per src
src_count = Counter(item['src'] for item in final_items)
total_items = len(final_items)
src_ratio = {src: count / total_items for src, count in src_count.items()}

# 5. Within each 'src' group, find the ratio of msg turn count
src_turn_count = defaultdict(list)
for item in final_items:
    turn_count = len(item.get('msg', []))  # msg의 길이로 turn 개수 계산
    src_turn_count[item['src']].append(turn_count)

src_turn_ratio = {src: Counter(turns) for src, turns in src_turn_count.items()}

print("\n++ Turn Count Ratio per Source:")
for src, counts in src_turn_ratio.items():
    total_turns = sum(counts.values())
    
    # Calculate ratios for each turn
    ratios = {turn: count / total_turns * 100 for turn, count in counts.items()}  # Convert to percentages

    # Check if turn 1 is included and its ratio is 100%
    if 1 in counts and ratios[1] == 100:
        continue

    print(f"\n{src}")
    for turn, count in counts.items():
        ratio = count / total_turns
        print(f"{turn} turns: {count}개({ratio:.2%})")

# 추가: eval_prediction.jsonl의 turn 개수 및 비율 출력
eval_prediction_file = './data/eval_prediction.jsonl'

# eval_prediction.jsonl 파일에서 turn 개수 및 비율 계산
turn_counts = Counter()
for item in final_items:
    turn_count = len(item.get('msg', []))  # 각 아이템의 msg 길이를 turn 개수로 사용
    if 1 <= turn_count <= 3:  # turn count가 1에서 3 사이인지 확인
        turn_counts[turn_count] += 1

total_turn_counts = sum(turn_counts.values())

print("\n++ Turn Counts in eval_prediction.jsonl:")
for turn, count in turn_counts.items():
    percentage = (count / total_turn_counts) * 100 if total_turn_counts > 0 else 0
    print(f"{turn} turns: {count}개({percentage:.2f}%)")

# Save merged results to a new JSONL file
with open(eval_prediction_file, 'w', encoding='utf-8') as f:
    for item in final_items:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Src Ratio Visualization (Pie Chart)
def plot_src_ratio(src_ratio):
    labels = list(src_ratio.keys())
    sizes = list(src_ratio.values())
    
    plt.figure(figsize=(6, 6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
    plt.title('Source Ratio')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

# 2. Turn Count Ratio per Source (Bar Chart)
def plot_turn_count_ratio(src_turn_ratio):
    for src, counts in src_turn_ratio.items():
        turns = list(counts.keys())
        ratios = [count / sum(counts.values()) * 100 for count in counts.values()]  # Convert ratios to percentages

        # Check if turn 1 is included and its ratio is 1
        if 1 in turns and ratios[turns.index(1)] == 100.0:  # Compare with 100% for clarity
            continue

        # Create a new figure for each source
        plt.figure(figsize=(2, 2))
        
        # Set the number of bars per row
        n_bars = 4
        n_rows = (len(turns) + n_bars - 1) // n_bars  # Calculate number of rows needed
        
        # Create subplots
        for i in range(n_rows):
            start_idx = i * n_bars
            end_idx = min(start_idx + n_bars, len(turns))

            # Create a barplot for the current row
            sns.barplot(x=[f'{turn} turn' for turn in turns[start_idx:end_idx]], 
                        y=ratios[start_idx:end_idx])

            # Add title and labels
            plt.title(src, fontsize=8)
            plt.xlabel('Number of Turns', fontsize=6)
            plt.ylabel('Ratio (%)', fontsize=6)
            plt.ylim(0, 100)  # Set y-axis between 0 and 100 for percentages
            
            # Display count above each bar
            for j, ratio in enumerate(ratios[start_idx:end_idx]):
                plt.text(j, ratio + 1, f'{counts[turns[start_idx + j]]}', ha='center', fontsize=8)

            plt.show()  # Show the plot for the current row

# Call the visualization functions
plot_src_ratio(src_ratio)
plot_turn_count_ratio(src_turn_ratio)

##### Test Set 가답안의 documents 제외

In [2]:
sigle_turn_synonyms_valid = f"./data/valid_1turn_normal_1019-0203.jsonl"
sigle_turn_normal_valid = f"./data/valid_1turn_synonyms_1019-0203.jsonl"
triple_turn_normal_valid = f"./data/valid_3turn_normal_1019-0203.jsonl"
sigle_turn_smalltalk_valid = f"./data/valid_1turn_smalltalk_1020-0244.jsonl"

In [ ]:
# eval.prediction.jsonl에서 docid를 추출하는 함수
def get_eval_docids(eval_file):
    docids = set()
    with open(eval_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            docids.add(data['docid'])
    return docids

# valid_1turn_normal_1019-0203.jsonl에서 eval.prediction.jsonl에 없는 docid만 필터링하는 함수
def filter_valid_file(valid_file, eval_docids, output):
    total_count = 0
    removed_count = 0
    kept_count = 0
    with open(valid_file, 'r', encoding='utf-8') as infile, open(output, 'w', encoding='utf-8') as outfile:
        for line in infile:
            total_count += 1
            data = json.loads(line)
            if data['docid'] in eval_docids:
                removed_count += 1  # 제거된 항목 수 증가
            else:
                kept_count += 1  # 남은 항목 수 증가
                outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
    
    return total_count, removed_count, kept_count

def generate_validset(eval_prediction_file, valid_file, output_file):
    eval_docids = get_eval_docids(eval_prediction_file)
    total_count, removed_count, kept_count = filter_valid_file(valid_file, eval_docids, output_file)

    # 결과 출력
    print(f"\n총 항목 수: {total_count}")
    print(f"제거된 항목 수: {removed_count}")
    print(f"남은 항목 수: {kept_count}")
    print(f"필터링된 결과가 {output_file}에 저장되었습니다.")

# 파일 경로
eval_prediction_file = './data/eval_prediction.jsonl'
print(f"gt: {eval_prediction_file}")

valid_file = './data/valid_1turn_normal_1019-0203.jsonl'
valid_1turn_normal_filtered = './data/valid_1turn_normal_1019-0203_filtered.jsonl'
generate_validset(eval_prediction_file, valid_file, valid_1turn_normal_filtered)

valid_file = './data/valid_1turn_synonyms_1019-0203.jsonl'
valid_1turn_synonyms_filtered = './data/valid_1turn_synonyms_1019-0203_filtered.jsonl'
generate_validset(eval_prediction_file, valid_file, valid_1turn_synonyms_filtered)

valid_file = './data/valid_3turn_normal_1019-0203.jsonl'
valid_3turn_normal_filtered = './data/valid_3turn_normal_1019-0203_filtered.jsonl'
generate_validset(eval_prediction_file, valid_file, valid_3turn_normal_filtered)

In [ ]:
# import json
# import random

# # 파일에서 JSONL 데이터를 읽는 함수
# def read_jsonl(file_path):
#     data = []
#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             data.append(json.loads(line))
#     return data

# # JSONL 데이터를 주어진 개수만큼 랜덤하게 뽑는 함수
# def sample_items(data, num_items):
#     return random.sample(data, num_items)

# # 파일과 비율을 전달받아 아이템을 랜덤으로 선택하고 합치는 함수
# def merge_random_selection(files_and_percentages, output_file):
#     merged_data = []
    
#     # 파일의 데이터와 개수 저장
#     file_data = {file_path: read_jsonl(file_path) for file_path, _ in files_and_percentages}
    
#     # 가장 작은 비율을 가진 파일의 이름과 데이터 추출
#     min_percentage_file = min(files_and_percentages, key=lambda x: x[1])
#     min_file_path, min_percentage = min_percentage_file
    
#     # 최소 비율 파일의 모든 데이터 포함
#     min_sample_size = len(file_data[min_file_path])
#     merged_data.extend(file_data[min_file_path])
#     print(f"{min_file_path}에서 뽑힌 아이템 수: {min_sample_size}")
    
#     # 나머지 파일에서 샘플링
#     for file_path, percentage in files_and_percentages:
#         if file_path != min_file_path:
#             # 나머지 파일에서 샘플링할 수량 계산
#             sample_size = int(min_sample_size * (percentage / min_percentage))
#             sample_size = min(sample_size, len(file_data[file_path]))  # 최대 개수 제한
#             sampled_data = sample_items(file_data[file_path], sample_size)
#             merged_data.extend(sampled_data)
            
#             # 각 파일에서 뽑힌 아이템 수 출력
#             print(f"{file_path}에서 뽑힌 아이템 수: {sample_size}")

#     # valid_id 필드 순서대로 업데이트
#     for idx, item in enumerate(merged_data):
#         item['valid_id'] = idx  # valid_id 필드를 아이템의 순서대로 업데이트
    
#     # 결과 저장
#     with open(output_file, 'w', encoding='utf-8') as outfile:
#         for item in merged_data:
#             outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

#     print(f"총 {len(merged_data)}개의 아이템이 {output_file}에 저장되었습니다.")

# files_and_percentages = [
#     (valid_3turn_normal_filtered, 0.05),  # 1번 파일에서 5%
#     (valid_1turn_normal_filtered, 0.25),  # 2번 파일에서 25%
#     (valid_1turn_synonyms_filtered, 0.75)   # 3번 파일에서 75%
# ]

# output_file = './data/valid_full.jsonl'
# merge_random_selection(files_and_percentages, output_file)

In [ ]:
# eval.jsonl에서 content를 추출하는 함수
def get_eval_docids(eval_file):
    docids = set()
    with open(eval_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            docids.add(data['docid'])
    return docids

# valid_1turn_normal_1019-0203.jsonl에서 eval.prediction.jsonl에 없는 docid만 필터링하는 함수
def filter_valid_file(valid_file, eval_docids, output):
    total_count = 0
    removed_count = 0
    kept_count = 0
    with open(valid_file, 'r', encoding='utf-8') as infile, open(output, 'w', encoding='utf-8') as outfile:
        for line in infile:
            total_count += 1
            data = json.loads(line)
            if data['docid'] in eval_docids:
                removed_count += 1  # 제거된 항목 수 증가
            else:
                kept_count += 1  # 남은 항목 수 증가
                outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
    
    return total_count, removed_count, kept_count

def generate_validset(eval_prediction_file, valid_file, output_file):
    eval_docids = get_eval_docids(eval_prediction_file)
    total_count, removed_count, kept_count = filter_valid_file(valid_file, eval_docids, output_file)

    # 결과 출력
    print(f"\n총 항목 수: {total_count}")
    print(f"제거된 항목 수: {removed_count}")
    print(f"남은 항목 수: {kept_count}")
    print(f"필터링된 결과가 {output_file}에 저장되었습니다.")

# 파일 경로
eval_prediction_file = './data/eval_prediction.jsonl'
print(f"gt: {eval_prediction_file}")

valid_file = './data/valid_1turn_normal_1019-0203.jsonl'
valid_1turn_normal_filtered = './data/valid_1turn_normal_1019-0203_filtered.jsonl'
generate_validset(eval_prediction_file, valid_file, valid_1turn_normal_filtered)

valid_file = './data/valid_1turn_synonyms_1019-0203.jsonl'
valid_1turn_synonyms_filtered = './data/valid_1turn_synonyms_1019-0203_filtered.jsonl'
generate_validset(eval_prediction_file, valid_file, valid_1turn_synonyms_filtered)

valid_file = './data/valid_3turn_normal_1019-0203.jsonl'
valid_3turn_normal_filtered = './data/valid_3turn_normal_1019-0203_filtered.jsonl'
generate_validset(eval_prediction_file, valid_file, valid_3turn_normal_filtered)

In [ ]:
import json
import random

# 파일에서 JSONL 데이터를 읽는 함수
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# JSONL 데이터를 주어진 개수만큼 랜덤하게 뽑는 함수
def sample_items(data, num_items):
    return random.sample(data, num_items)

# 파일과 비율을 전달받아 아이템을 랜덤으로 선택하고 합치는 함수
def merge_random_selection(files_and_percentages, total_items, output_file):
    merged_data = []
    
    # 파일의 데이터와 개수 저장
    file_data = {file_path: read_jsonl(file_path) for file_path, _ in files_and_percentages}
    
    # 가장 작은 비율을 가진 파일 추출
    min_percentage_file = min(files_and_percentages, key=lambda x: x[1])
    min_file_path, min_percentage = min_percentage_file
    
    # 각 파일에서 샘플링할 수량 계산
    for file_path, percentage in files_and_percentages:
        # 총 아이템 개수에 따라 샘플링 수량 계산
        sample_size = int(total_items * percentage)
        
        # 만약 해당 파일의 아이템 수가 부족하면 최소 퍼센트 기준으로 조정
        if len(file_data[file_path]) < sample_size:
            sample_size = min(len(file_data[file_path]), int(total_items * min_percentage))
        
        # 아이템 샘플링
        sampled_data = sample_items(file_data[file_path], sample_size)
        merged_data.extend(sampled_data)
        
        # 각 파일에서 뽑힌 아이템 수 출력
        print(f"{file_path}에서 뽑힌 아이템 수: {sample_size}")

    # valid_id 필드 순서대로 업데이트
    for idx, item in enumerate(merged_data):
        item['valid_id'] = idx  # valid_id 필드를 아이템의 순서대로 업데이트
    
    # 결과 저장
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for item in merged_data:
            outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f"총 {len(merged_data)}개의 아이템이 {output_file}에 저장되었습니다.\n")



valid_1turn_normal_filtered = './data/valid_1turn_normal_1019-0203_filtered.jsonl'
valid_1turn_synonyms_filtered = './data/valid_1turn_synonyms_1019-0203_filtered.jsonl'
valid_3turn_normal_filtered = './data/valid_3turn_normal_1019-0203_filtered.jsonl'

files_and_percentages = [
    ("./data/valid_1turn_smalltalk_1020-0244.jsonl", 0.10),   # 1턴 일상대화 
    (valid_1turn_synonyms_filtered, 0.50),   # 1턴 유사어 질의
    (valid_1turn_normal_filtered, 0.30),  # 1턴 일반 질의
    (valid_3turn_normal_filtered, 0.10),  # 3턴 일반 질의
]

total_items = 230  # 최종 아이템 개수
output_file = f'./data/valid_{total_items}.jsonl'
merge_random_selection(files_and_percentages, total_items, output_file)

total_items = 460  # 최종 아이템 개수
output_file = f'./data/valid_{total_items}.jsonl'
merge_random_selection(files_and_percentages, total_items, output_file)

total_items = 1000  # 최종 아이템 개수
output_file = f'./data/valid_{total_items}.jsonl'
merge_random_selection(files_and_percentages, total_items, output_file)

In [ ]:
import json
import random

# 파일에서 JSONL 데이터를 읽는 함수
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# JSONL 데이터를 주어진 개수만큼 랜덤하게 뽑는 함수
def sample_items(data, num_items):
    return random.sample(data, num_items)

# "docid"를 "topk"로 변환하는 함수
def convert_docid_to_topk(data):
    for item in data:
        if 'docid' in item:
            item['topk'] = [item.pop('docid')]  # "docid"를 "topk" 배열로 변환
    return data

# 파일과 비율을 전달받아 아이템을 랜덤으로 선택하고 합치는 함수
def merge_random_selection(files_and_percentages, total_items, output_file):
    merged_data = []
    
    # 파일의 데이터와 개수 저장
    file_data = {file_path: read_jsonl(file_path) for file_path, _ in files_and_percentages}
    
    # 가장 작은 비율을 가진 파일 추출
    min_percentage_file = min(files_and_percentages, key=lambda x: x[1])
    min_file_path, min_percentage = min_percentage_file
    
    # 각 파일에서 샘플링할 수량 계산
    for file_path, percentage in files_and_percentages:
        # 총 아이템 개수에 따라 샘플링 수량 계산
        sample_size = int(total_items * percentage)
        
        # 만약 해당 파일의 아이템 수가 부족하면 최소 퍼센트 기준으로 조정
        if len(file_data[file_path]) < sample_size:
            sample_size = min(len(file_data[file_path]), int(total_items * min_percentage))
        
        # 아이템 샘플링
        sampled_data = sample_items(file_data[file_path], sample_size)
        
        # "docid"를 "topk"로 변환
        converted_data = convert_docid_to_topk(sampled_data)
        
        # 변환된 데이터를 병합
        merged_data.extend(converted_data)
        
        # 각 파일에서 뽑힌 아이템 수 출력
        print(f"{file_path}에서 뽑힌 아이템 수: {sample_size}")

    # valid_id 필드 순서대로 업데이트
    for idx, item in enumerate(merged_data):
        item['valid_id'] = idx  # valid_id 필드를 아이템의 순서대로 업데이트
    
    # 결과 저장
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for item in merged_data:
            outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f"총 {len(merged_data)}개의 아이템이 {output_file}에 저장되었습니다.\n")

valid_1turn_normal_filtered = './data/valid_1turn_normal_1019-0203_filtered.jsonl'
valid_1turn_synonyms_filtered = './data/valid_1turn_synonyms_1019-0203_filtered.jsonl'
valid_3turn_normal_filtered = './data/valid_3turn_normal_1019-0203_filtered.jsonl'

files_and_percentages = [
    ("./data/valid_1turn_smalltalk_1020-0244.jsonl", 0.10),   # 1턴 일상대화 
    (valid_1turn_synonyms_filtered, 0.50),   # 1턴 유사어 질의
    (valid_1turn_normal_filtered, 0.30),  # 1턴 일반 질의
    (valid_3turn_normal_filtered, 0.10),  # 3턴 일반 질의
]

total_items = 230  # 최종 아이템 개수
output_file = f'./data/valid_{total_items}.jsonl'
merge_random_selection(files_and_percentages, total_items, output_file)

total_items = 460  # 최종 아이템 개수
output_file = f'./data/valid_{total_items}.jsonl'
merge_random_selection(files_and_percentages, total_items, output_file)

total_items = 1000  # 최종 아이템 개수
output_file = f'./data/valid_{total_items}.jsonl'
merge_random_selection(files_and_percentages, total_items, output_file)

### topk 2,3 채우기

In [3]:
import json

def process_files(a_file, b_file, result_file):
    # 파일에서 데이터 읽기
    with open(a_file, 'r', encoding='utf-8') as f:
        a_data = [json.loads(line) for line in f]
    
    with open(b_file, 'r', encoding='utf-8') as f:
        b_data = [json.loads(line) for line in f]
    
    # 결과를 저장할 리스트
    result_data = []
    
    # a_data의 각 항목에 대해 처리
    for a_item in a_data:
        # b_data에서 matching eval_id 찾기
        b_item = next((item for item in b_data if item['eval_id'] == a_item['eval_id']), None)
        
        if b_item:
            # 새로운 topk 리스트 생성
            new_topk = a_item['topk']
            for id in b_item['topk']:
                if id not in new_topk:
                    new_topk.append(id)
                    if len(new_topk) == 3:  # 최대 3개까지만 추가
                        break
            
            # 새로운 항목 생성
            new_item = {
                'eval_id': a_item['eval_id'],
                'msg': a_item['msg'],
                'topk': new_topk
            }
            
            result_data.append(new_item)
    
    # 결과를 파일에 쓰기
    with open(result_file, 'w', encoding='utf-8') as f:
        for item in result_data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

# 파일 경로
jsonl_file_1 = './data/valid_230_topk1.jsonl'
jsonl_file_2 = './output/1021-1444_topk2.csv' 
output_file = './data/valid_230.jsonl'  # 결과를 저장할 파일 경로

# 함수 실행
process_files(jsonl_file_1, jsonl_file_2, output_file)

In [ ]:
import json

def check_duplicate_topk(jsonl_file):
    # 중복 항목을 저장할 리스트
    duplicates = []
    
    with open(jsonl_file, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            topk_items = data.get("topk", [])
            
            # 중복 체크를 위한 세트 생성
            seen = set()
            for item in topk_items:
                if item in seen:
                    duplicates.append(data["eval_id"])  # eval_id를 중복 리스트에 추가
                    break  # 중복이 발견되면 더 이상 체크하지 않음
                seen.add(item)

    if duplicates:
        print("중복 항목이 발견된 eval_id:", set(duplicates))  # 중복 eval_id 출력
    else:
        print("중복 항목이 없습니다.")

# 사용 예시
jsonl_file = './data/valid_230_fill_topk3.jsonl'  # 체크할 JSONL 파일 경로
check_duplicate_topk(jsonl_file)

In [ ]:
import json

def calculate_topk_distribution(jsonl_file):
    # 각 topk 항목 수에 대한 카운터 초기화
    counts = {1: 0, 2: 0, 3: 0}
    total_items = 0
    
    with open(jsonl_file, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            topk_items = data.get("topk", [])
            topk_count = len(topk_items)  # topk 항목 수

            if topk_count in counts:
                counts[topk_count] += 1
            total_items += 1  # 전체 아이템 카운트 증가

    # 비율 계산
    if total_items > 0:
        distribution = {k: (v / total_items) * 100 for k, v in counts.items()}
    else:
        distribution = {1: 0, 2: 0, 3: 0}

    # 결과 출력
    print(f"전체 아이템 수: {total_items}")
    for k in range(1, 4):
        print(f"topk 항목이 {k}개인 비율: {distribution.get(k, 0):.2f}%")

# 사용 예시
jsonl_file = './data/valid_230_fill_topk3.jsonl'  # 분석할 JSONL 파일 경로
calculate_topk_distribution(jsonl_file)

jsonl_file = './data/valid_230.jsonl'  # 분석할 JSONL 파일 경로
calculate_topk_distribution(jsonl_file)